#CONEXIÓN Y LECTURA BD

## Conexión BD

In [1]:
# Montar drive y crear path para sys y para directorio de trabajo

from google.colab import drive
drive.mount('/content/drive')

import sys
import os

# Define el path al directorio de trabajo actual
path = "/content/drive/My Drive/cod/analitica_salud"


sys.path.append(path) ### para importar archivo de funciones propias a través de import

os.chdir(path) ### para que por defecto suba y descargue archivos partiendo de esa ruta

Mounted at /content/drive


## Cargar Librerías

In [7]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

##Acceso Notebook

In [5]:
salud_model= pd.read_csv('data/salud_modelado_final.csv')
salud_model.head()

,cholesterol_2,cholesterol_3,gluc_2,gluc_3,smoke,alco,active,age,height,weight,ap_hi,ap_lo,cardio
0,0,0,0,0,0,0,1,50,168,62.0,110,80,0
1,0,1,0,0,0,0,1,55,156,85.0,140,90,1
2,0,1,0,0,0,0,0,51,165,64.0,130,70,1
3,0,0,0,0,0,0,1,48,169,82.0,150,100,1
4,0,0,0,0,0,0,0,47,156,56.0,100,60,0


# Modelo

## Regresión Logística

In [8]:
x = salud_model.drop("cardio", axis=1)

y = salud_model["cardio"]

x_train,x_test,y_train,y_test = train_test_split(x,y,shuffle = True,test_size=0.30, random_state=123)


model = LogisticRegression() # definir el modelo
model.fit(x_train,y_train) # entrenar el modelo

y_pred_train = model.predict(x_train) # guardar la predicción para train
y_pred_test = model.predict(x_test) # guardar la predicción para test


y_pred_test # predicción de test
y_test # realidad de test

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,cardio
52751,1
28203,0
42946,0
45738,0
5086,0
...,...
6471,1
21495,1
13220,1
18854,1


### Metricas de Desempeño

## Árbol de Decisión

### Métricas de Desempeño

# Optimización de Hiperparámetros